In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px 
import geopandas as gpd
from branca.colormap import LinearColormap
import folium
import folium.plugins as plugins

In [6]:
asylum = pd.read_csv('/Users/ignaciolazarovalganon/Downloads/GitHub/Refugees-flow-study-project/Data/asylum_decisions_CLEAN.csv')
#asylum = pd.read_csv('Data/asylum_decisions_CLEAN.csv', header=13)

In [9]:
asylum.head()

,Year,Country of origin,Country of origin (ISO),Country of asylum,Country of asylum (ISO),Authority,Stage of procedure,Cases / Persons,Recognized decisions,Complementary protection,Rejected decisions,Otherwise closed,Total decisions
0,2014,Ukraine,UKR,Russian Federation,RUS,G,TA,P,0,248201,0,16687,264888
1,2015,Ukraine,UKR,Russian Federation,RUS,G,TA,P,0,148379,0,21,148400
2,2022,Syrian Arab Rep.,SYR,Germany,DEU,G,FI,P,15078,51754,34,6466,73332
3,2016,Syrian Arab Rep.,SYR,Sweden,SWE,G,FI,P,2504,40971,136,4035,47646
4,2020,Venezuela (Bolivarian Republic of),VEN,Spain,ESP,G,FI,P,5,40396,882,0,41283


In [11]:
mapa_origin = gpd.read_file("Data/countries.geojson")
mapa_origin.rename(columns={'ISO_A3': 'Country of origin (ISO)', 'ADMIN':'Country'}, inplace=True)
mapa_origin


DriverError: Data/countries.geojson: No such file or directory

In [ ]:
pop1 = population[['Country of origin (ISO)', 'Total forced displacements','Refugees under UNHCR\'s mandate', 'Asylum-seekers', 'IDPs of concern to UNHCR', ]].groupby('Country of origin (ISO)').sum().reset_index()



In [ ]:
mapa_origin = pd.merge(mapa_origin, pop1, on='Country of origin (ISO)', how='inner')
map_dict = pop1.set_index('Country of origin (ISO)')['Total forced displacements'].to_dict()

color_scale = LinearColormap(['yellow','red'], vmin=min(map_dict.values()), vmax=max(map_dict.values()))
def get_color(feature):
    country_name = feature['properties']['Country of origin (ISO)']
    value = map_dict.get(country_name)
    if value is None:
        return 'gray'  # Assign a default color for None values
    return color_scale(value)



map1 = folium.Map(location=[52.3680, 4.9036], zoom_start=3)

folium.GeoJson(data=mapa_origin, 
            name='Origin',
            tooltip=folium.features.GeoJsonTooltip(fields=['Country','Total forced displacements', 'Refugees under UNHCR\'s mandate', 'Asylum-seekers','IDPs of concern to UNHCR' ],labels=True, sticky=False),
                style_function= lambda feature: {
                    'fillColor': get_color(feature),
                    'color': 'black',
                    'weight': 1,
                    'dashArray': '5, 5',
                    'fillOpacity':0.5},
                highlight_function=lambda feature: {'weight':3, 'fillColor': get_color(feature), 'fillOpacity': 1}).add_to(map1) 

map1

In [ ]:
#Mapa de países de asilo 
mapa_asylum = gpd.read_file(r"C:\Users\ioana\OneDrive\Escritorio\Entorno10\Refugee_project_study\Refugees-flow-study-project\Data\countries (1).geojson") #leemos el geojsom
mapa_asylum.rename(columns={'ISO_A3': 'Country of asylum (ISO)', 'ADMIN':'Country'}, inplace=True) #renombramos la columna ISO_A3 a Country of asylum (ISO) para poder hacer el merge con population
#hacemos un df de asilo con los datos de pais de asilo ISO, total forced displacements, refugees under UNHCR's mandate, asylum-seekers, IDPs of concern to UNHCR
pop2 = population[['Country of asylum (ISO)', 'Total forced displacements','Refugees under UNHCR\'s mandate', 'Asylum-seekers', 'IDPs of concern to UNHCR']].groupby('Country of asylum (ISO)').sum().reset_index()
#mergeamos el mapa con el df de asilo
mapa_asylum = pd.merge(mapa_asylum, pop2, on='Country of asylum (ISO)', how='inner')

#personalizamos el color en funcion del total forced displacements
map_dict = pop2.set_index('Country of asylum (ISO)')['Total forced displacements'].to_dict()
color_scale = LinearColormap(['yellow','green'], vmin=min(map_dict.values()), vmax=max(map_dict.values()))
def get_color(feature):
    
    country_name = feature['properties']['Country of asylum (ISO)']
    value = map_dict.get(country_name)
    if value is None:
        return 'gray'  
    return color_scale(value)

map2 = folium.Map(location=[52.3680, 4.9036], zoom_start=3)
#creamos el mapa con folium
folium.GeoJson(data=mapa_asylum, 
            name='Asylum',
            tooltip=folium.features.GeoJsonTooltip(fields=['Country','Total forced displacements', 'Refugees under UNHCR\'s mandate', 'Asylum-seekers','IDPs of concern to UNHCR' ],labels=True, sticky=False),
                style_function= lambda feature: {
                    'fillColor': get_color(feature),
                    'color': 'black',
                    'weight': 1,
                    'dashArray': '5, 5',
                    'fillOpacity':0.5},
                highlight_function=lambda feature: {'weight':3, 'fillColor': get_color(feature), 'fillOpacity': 1}).add_to(map2) 

map2